# import, data load

In [1]:
COLUMNS = ["user_id" ,"trip", "course", "place_id","place_name", "place_category","rating", "review", "place_address", "place_lat", "place_lng","y", "belief", "y_pred"]

In [2]:
!pip install haversine

In [3]:
import pandas as pd
import numpy as np
from haversine import haversine
import itertools

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
wd_0 = pd.read_csv('/content/drive/MyDrive/csv_result_data/recommend_li_couple_after.csv', names = COLUMNS, header=None, skiprows = 1)
deepFM_0 = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/couple_b.csv', names = COLUMNS, header=None, skiprows = 1)

wd_data_family = pd.read_csv('/content/drive/MyDrive/csv_result_data/recommend_li_family_after.csv', names = COLUMNS, header=None, skiprows = 1)
deepFM_data_family =  pd.read_csv('/content/drive/MyDrive/deepFM_result_data/family_b.csv', names = COLUMNS, header=None, skiprows = 1)

wd_data_hotel = pd.read_csv('/content/drive/MyDrive/csv_result_data/recommend_li_hotel_after.csv', names = COLUMNS, header=None, skiprows = 1)
deepFM_data_hotel = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/hotel_b.csv', names = COLUMNS, header=None, skiprows = 1)

wd_data_guesthouse = pd.read_csv('/content/drive/MyDrive/csv_result_data/recommend_li_guesthouse_after.csv', names = COLUMNS, header=None, skiprows = 1)
deepFM_data_guesthouse = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/guesthouse_b.csv', names = COLUMNS, header=None, skiprows = 1)

wd_data_pension = pd.read_csv('/content/drive/MyDrive/csv_result_data/recommend_li_pension_after.csv', names = COLUMNS, header=None, skiprows = 1)
deepFM_data_pension = pd.read_csv('/content/drive/MyDrive/deepFM_result_data/pension_b.csv', names = COLUMNS, header=None, skiprows = 1)

In [12]:
wd_data_couple = wd_0.fillna(wd_0['place_lat'].mean())
wd_data_couple.loc[221]

deepFM_data_couple = deepFM_0.fillna(deepFM_0['place_lat'].mean())
deepFM_data_couple.loc[221]

user_id                           446
trip                           couple
course                              2
place_id                           96
place_name                  강릉정동진해안단구
place_category                  관광,명소
rating                              0
review                              0
place_address     강원 강릉시 강동면 심곡리 1-11
place_lat                     37.6663
place_lng                     129.055
y                                   0
belief                              0
y_pred                       0.122043
Name: 221, dtype: object

# 함수

In [13]:
def min_distance(user_number, data):
  loc_1 = (data.loc[user_number]['place_lat'], data.loc[user_number]['place_lng'])
  loc_2 = (data.loc[user_number+1]['place_lat'], data.loc[user_number+1]['place_lng'])
  loc_3 = (data.loc[user_number+2]['place_lat'], data.loc[user_number+2]['place_lng'])
  loc_4 = (data.loc[user_number+3]['place_lat'], data.loc[user_number+3]['place_lng'])
  loc_5 = (data.loc[user_number+4]['place_lat'], data.loc[user_number+4]['place_lng'])
  result = list(itertools.permutations([loc_1, loc_2, loc_3, loc_4, loc_5], 5))
  result_str = list(itertools.permutations(["1", "2", "3", "4", "5"], 5))
  min_dis = 0
  course = []
  n = len(result)
  for i in range(0, n):
    dis = haversine(result[i][0],result[i][1]) + haversine(result[i][1],result[i][2]) + haversine(result[i][2],result[i][3]) + haversine(result[i][3],result[i][4])
    if(i==0):
      min_dis = dis
    if(dis < min_dis):
      min_dis = dis
      course = result_str[i]
  return min_dis , course

In [14]:
def get_distance_table(data_before, data_after):
  distance_dataframe = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
  diff_accumu = 0
  for i in range(0, 50):
    user_index = i * 5 
    before_dis, before_course = min_distance(user_index, data_before)
    before_course_str = " ".join(before_course)
    after_dis, after_course = min_distance(user_index, data_after)
    after_course_str = " ".join(after_course)
    distance_dataframe = distance_dataframe.append({'user_id' : str(data_before.loc[user_index]['user_id'])}, ignore_index=True)
    distance_dataframe.loc[i]['wide_and_deep'] = before_dis
    distance_dataframe.loc[i]['wide_and_deep_course'] = before_course_str
    distance_dataframe.loc[i]['deepFM'] = after_dis
    distance_dataframe.loc[i]['deepFM_course'] = after_course_str
    diff = before_dis - after_dis
    distance_dataframe.loc[i]['diff'] = diff
    ratio = after_dis/before_dis
    distance_dataframe.loc[i]['ratio'] = ratio
    diff_accumu = diff_accumu + diff
    distance_dataframe.loc[i]['diff_accumulate'] = diff_accumu 

  return distance_dataframe

# 카테고리별 거리 표 구하기

In [21]:
couple_distance_table = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
couple_distance_table = get_distance_table(wd_data_couple, deepFM_data_couple)

couple_distance_table.to_csv('/content/drive/MyDrive/addr_RESULT/couple.xlsx',index=False)

In [22]:
family_distance_table = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
family_distance_table = get_distance_table(wd_data_family, deepFM_data_family)
family_distance_table.to_csv('/content/drive/MyDrive/addr_RESULT/family.xlsx',index=False)

In [23]:
hotel_distance_table = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
hotel_distance_table = get_distance_table(wd_data_hotel, deepFM_data_hotel)
hotel_distance_table.to_csv('/content/drive/MyDrive/addr_RESULT/hotel.xlsx',index=False)

In [24]:
guesthouse_distance_table = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
guesthouse_distance_table = get_distance_table(wd_data_guesthouse, deepFM_data_guesthouse)
guesthouse_distance_table.to_csv('/content/drive/MyDrive/addr_RESULT/guesthouse.xlsx',index=False)

In [25]:
pension_distance_table = pd.DataFrame(columns=['user_id',	'wide_and_deep', 'wide_and_deep_course',	'deepFM', "deepFM_course", "diff",'ratio', "diff_accumulate"])
pension_distance_table = get_distance_table(wd_data_pension, deepFM_data_pension)
pension_distance_table.to_csv('/content/drive/MyDrive/addr_RESULT/pension.xlsx',index=False)